In [2]:
import pandas as pd

In [7]:
commodities = pd.read_csv('../data/commodity/scraped-commodities.csv', dtype={'monthyear':str,
                                                                              'lock_id':str,
                                                                              'comm_code':str})
lock_info = pd.read_csv('../data/commodity/lock_codes.csv', dtype={'Lock No':str})
lock_info = lock_info[lock_info['River Code'] == 'OH']

In [8]:
print(lock_info)

    River Code  River Name Lock No                       Lock Name
134         OH  OHIO RIVER      21        BELLEVILLE LOCKS AND DAM
135         OH  OHIO RIVER      75          CANNELTON LOCK AND DAM
136         OH  OHIO RIVER      25  CAPT ANT MELDAHL LOCKS AND DAM
137         OH  OHIO RIVER      02          DASHIELDS LOCK AND DAM
138         OH  OHIO RIVER      01           EMSWORTH LOCK AND DAM
139         OH  OHIO RIVER      24           GREENUP LOCKS AND DAM
140         OH  OHIO RIVER      71          HANNIBAL LOCKS AND DAM
141         OH  OHIO RIVER      77        JOHN T. MYERS LOCK & DAM
142         OH  OHIO RIVER      41          MARKLAND LOCKS AND DAM
143         OH  OHIO RIVER      42          MCALPINE LOCKS AND DAM
144         OH  OHIO RIVER      03         MONTGOMERY LOCK AND DAM
145         OH  OHIO RIVER      04     NEW CUMBERLAND LOCK AND DAM
146         OH  OHIO RIVER      76           NEWBURGH LOCK AND DAM
147         OH  OHIO RIVER      79           OLMSTED LOCKS AND

In [6]:
print(commodities.sort_values('total_tons',ascending=False).head(50))

     monthyear lock_id comm_code                                comm_desc  \
1462    052019      22        10                   Coal, Lignite And Coke   
357     102018      22        10                   Coal, Lignite And Coke   
49      082018      22        10                   Coal, Lignite And Coke   
1514    052019      71        10                   Coal, Lignite And Coke   
1141    032019      22        10                   Coal, Lignite And Coke   
1454    052019      21        10                   Coal, Lignite And Coke   
832     012019      22        10                   Coal, Lignite And Coke   
348     102018      21        10                   Coal, Lignite And Coke   
411     102018      71        10                   Coal, Lignite And Coke   
1132    032019      21        10                   Coal, Lignite And Coke   
41      082018      21        10                   Coal, Lignite And Coke   
99      082018      71        10                   Coal, Lignite And Coke   